In [12]:
#개봉 2일후, 4일전 까지의 데이터를 가지고, 개봉 7일 후, 13일 후 누적관객수를 예측하기
#따라서, 데이터는 개봉 전 데이터를 가지고 예측을 해야함, 개봉 후 데이터로 예측해야하는 것은 휴일의 수

import pandas as pd
import numpy as np
import math
import datetime
import os
import sys
import cufflinks as cf    
import matplotlib.pyplot as plt
import re

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
cf.go_offline()


pd.set_option('max_columns' , 999)
pd.set_option('max_rows' , 999)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [13]:
def edit_File(lst) :
    new_lst = []
    
    for item in lst :
        new_item = item[:-4]
        new_lst.append(new_item)
        
    return new_lst
            

In [14]:
def renameCol(df,date) :
    df = df.rename(columns = {"기간별 박스오피스 검색 리스트" : "순위", "Unnamed: 1" : "영화명", "Unnamed: 2" : "개봉일", "Unnamed: 3" : "매출액", "Unnamed: 4" : "매출액 점유율", 
                    "Unnamed: 5" : "누적매출액", "Unnamed: 6" : "관객수", "Unnamed: 7" : "누적관객수", "Unnamed: 8" : "스크린수", "Unnamed: 9" : "상영횟수", "Unnamed: 10" : "대표국적", 
                    "Unnamed: 11" : "국적", "Unnamed: 12" : "제작사", "Unnamed: 13" : "배급사", "Unnamed: 14" : "등급", "Unnamed: 15" : "장르", "Unnamed: 16" : "감독", "Unnamed: 17" : "배우"})
    #tail = df.tail(1)
    df = df[3:-1]
    df = df.set_index("영화명", drop=True)
    
    
    
    for col in ["순위", "매출액", "누적매출액", "관객수", "누적관객수", "스크린수", "상영횟수"] :
             df[col] = df[col].apply(lambda x: int(x.replace(',', '')))
    df["매출액 점유율"] = df["매출액 점유율"].apply(lambda x: float(x[:-1].replace(".", "")))
    
    for col in ["관객수", "스크린수", "상영횟수"] :
             df[col] = df[col].apply(lambda x: int(str(x).replace(',', '')))
            
    
    
    #다중공산성 문제 고려해보기
    #df["스크린수 점유율"] = format(int(df["스크린수"].tolist()[0]) / int(tail["스크린수"].tolist()[0]), ".1f")
    #df["관객수 점유율"] = format(int(df["관객수"].tolist()[0]) / int(tail["관객수"].tolist()[0]), ".1f")
    #df["상영횟수 점유율"] = format(int(df["상영횟수"].tolist()[0]) / int(tail["상영횟수"].tolist()[0]), ".1f")
    
    df = making_month_col(df, date)
    
    return df

In [15]:
def renameCol2(df) :
    dic = {"개봉일람 검색" : "순번"}
    string = "Unnamed: "
    for idx, col in zip(list(range(1, 18)), ["영화명", "감독", "제작사", "수입사", "배급사", "개봉일", "영화유형", "영화형태", "대표국적", "전국스크린수", "전국매출액", "전국관객수", "서울매출액", "서울관객수", "장르", "등급", "영화구분"]) :
        dic[string + str(idx)] = col
    
    df= df.rename(columns = dic)[2:]
    df = df.set_index("영화명", drop=True)
    
    #날짜구분
    df["개봉일"] = df["개봉일"].map(lambda x : datetime.datetime.strptime(str(x), '%Y-%m-%d'))
    df["year"] = df["개봉일"].map(lambda x : int(x.timetuple().tm_year))
    df["month"] = df["개봉일"].map(lambda x : int(x.timetuple().tm_mon))
    df["day"] = df["개봉일"].map(lambda x : int(x.timetuple().tm_mday))
    df["weekday"] = df["개봉일"].map(lambda x: int(x.weekday()))
    
    df = df[["감독", "제작사", "수입사", "배급사", "개봉일", "대표국적", "장르", "등급", "year", "month", "day", "weekday", "영화구분"]]
    
    return df

In [16]:
"""
def making_month_col(df, date) :
    df
    datestr = "20" + date[0:2] + "-" + date[2:4] + "-" +date[4:]
    datestr = datetime.datetime.strptime(datestr, '%Y-%m-%d')
    
    #임시조치
    df = df[df["개봉일"] >"2013-01-01"]
    df["개봉일"] = df["개봉일"].fillna("2017-12-13")
    
    #날짜구분
    df["개봉일"] = df["개봉일"].map(lambda x : datetime.datetime.strptime(str(x), '%Y-%m-%d'))
    df["year"] = df["개봉일"].map(lambda x : int(x.timetuple().tm_year))
    df["month"] = df["개봉일"].map(lambda x : int(x.timetuple().tm_mon))
    df["day"] = df["개봉일"].map(lambda x : int(x.timetuple().tm_mday))

    before_28_after_14 = []
    
    for idx in range(-28, 15, 1) :
        new_daystr = datestr + datetime.timedelta(days=idx)
        new_daystr = new_daystr.strftime('%Y-%m-%d')
        before_28_after_14.append(new_daystr)
    
    #개봉일이 데이터 수집 1달전~2주후에 있는 것들만 수집
    df = df[df["개봉일"].isin(before_28_after_14)]
   
    for _ in ["4주전 ", "3주전 ", "2주전 ", "1주전 ", "1주후 ", "2주후 "] :
        for __ in ["순위", "매출액", "매출액 점유율", "누적매출액", "관객수", "누적관객수", "스크린수", "상영횟수"] :
            df[_ + __] = np.nan
   
    if list(df.index) :
        string =0
        
        for name, opendate in zip(df.index.tolist() ,df["개봉일"].tolist()) :
            
            day =  datestr - opendate
            day = int(day.days)
            if day >= 7 : 
                string = "2주후 "
            elif day >= 0 :
                string = "1주후 "
            elif day >= -7 :
                string = "1주전 "
            elif day >= -14 :
                string = "2주전 "
            elif day >= -21 :
                string = "3주전 "
            elif day >= -28 :
                string = "4주전 "
            
            for col in ["순위", "매출액", "매출액 점유율", "누적매출액", "관객수", "누적관객수", "스크린수", "상영횟수"] :
                #df.set_value(name, string+col , df[col][name])
                df.loc[name, string+col] =  df.loc[name, col]
                
    df = df.drop(["순위", "매출액", "매출액 점유율", "누적매출액", "관객수", "누적관객수", "스크린수", "상영횟수"], axis=1)
    
    return df
            
        
           
"""
"pass"

'pass'

In [17]:
def making_list_dummy(df, cols) :
    new_df = df.copy()
    for col in cols :
        
        dic = pd.Series(new_df[col].values,index=new_df.index).to_dict()
        s = pd.Series(dic)
        temp = pd.get_dummies(s.apply(pd.Series).stack(), prefix=col).sum(level=0)
        
        new_df = pd.concat([new_df, temp], axis=1)
        
    new_df = new_df.drop(cols, axis=1)
    
    return new_df

In [18]:
"""
#주어진 데이터 처리(새로운 데이터로 대체하므로 실행하지 않음)
path_dir = "C:\\Users\\LG_\\Desktop\\와이빅타\\17-2 빅콘-영화분석\\csv"
file_list = os.listdir(path_dir)
file_list.sort()

file_date = edit_File(file_list)
"""
"pass"

'pass'

In [19]:
"""
flag = True
for name, date in zip(file_list, file_date) :
    temp = pd.read_csv("csv/"+name, encoding='CP949')
    temp2 = renameCol(temp, date)
    
    if flag :
        new_df = temp2
        flag = False
    else :
        new_df = pd.concat([new_df, temp2[~temp2.index.isin(new_df.index)]])
        new_df.update(temp2)
"""
"pass"

'pass'

In [20]:
movie = pd.read_csv("movie_data/개봉작14~17.csv", encoding="CP949", engine="python")

In [21]:
new_df= renameCol2(movie)

In [22]:
#중복데이터 처리
d_lst = new_df.index.get_duplicates()

if d_lst :
    new_df.loc[d_lst, :]

In [23]:
#기타 전처리
new_df["runtime"] = np.nan
new_df["배우"] = np.nan
new_df["D+7 휴일수"] = np.nan
new_df["D+13 휴일수"] = np.nan

new_df["수입사"] = new_df["수입사"].str.split(",")
new_df["제작사"] = new_df["제작사"].str.split(",")
new_df["감독"] = new_df["감독"].str.split(",")
new_df["배급사"] = new_df["배급사"].str.split(",")
new_df["장르"] = new_df["장르"].str.split(",")
new_df["국산"] = new_df["대표국적"].map(lambda x: 1 if x =="한국" else 0 )
new_df["미국산"] = new_df["대표국적"].map(lambda x: 1 if x =="미국" else 0 )
new_df["영화구분"] = new_df["영화구분"].map(lambda x: 1 if x =="상업영화" else 0 )
new_df.head()

,감독,제작사,수입사,배급사,개봉일,대표국적,장르,등급,year,month,day,weekday,영화구분,runtime,배우,D+7 휴일수,D+13 휴일수,국산,미국산
영화명,,,,,,,,,,,,,,,,,,,
명량,[김한민],[(주)빅스톤픽쳐스],NaN,[씨제이이앤엠(주)],2014-07-30,한국,[사극],15세이상관람가,2014,7,30,2,1,NaN,NaN,NaN,NaN,1,0
국제시장,[윤제균],"[(주)제이케이필름, 씨제이이앤엠(주)]",NaN,[씨제이이앤엠(주)],2014-12-17,한국,[드라마],12세이상관람가,2014,12,17,2,1,NaN,NaN,NaN,NaN,1,0
베테랑,[류승완],[(주)외유내강],NaN,[씨제이이앤엠(주)],2015-08-05,한국,[액션],15세이상관람가,2015,8,5,2,1,NaN,NaN,NaN,NaN,1,0
암살,[최동훈],[(주)케이퍼필름],NaN,[(주)쇼박스],2015-07-22,한국,[액션],15세이상관람가,2015,7,22,2,1,NaN,NaN,NaN,NaN,1,0
부산행,[연상호],[(주)영화사레드피터],NaN,[(주)넥스트엔터테인먼트월드(NEW)],2016-07-20,한국,[액션],15세이상관람가,2016,7,20,2,1,NaN,NaN,NaN,NaN,1,0


In [24]:
#런타임 처리
runtime = pd.read_csv("movie_data/runtime.csv", encoding="CP949")
runtime = runtime[["name", "runtime"]].rename(columns = {"name" : "영화명"})
runtime = runtime.set_index(["영화명"])

runtime["runtime"] = runtime.runtime.map(lambda x : int(x[:-1]))
new_df.update(runtime)

In [25]:
#배우 처리
actors = pd.read_csv("movie_data/new_actors.csv", encoding="CP949")
actors = actors[["영화명","배우"]]
actors = actors.set_index(["영화명"])


actors = actors.drop_duplicates()
new_df.update(actors)

In [26]:
#휴일수 처리
y2014 = pd.read_csv("holiday/2014.csv", encoding = "CP949")
y2015 = pd.read_csv("holiday/2015.csv", encoding = "CP949")
y2016 = pd.read_csv("holiday/2016.csv", encoding = "CP949")
y2017 = pd.read_csv("holiday/2017.csv", encoding = "CP949")

col_lst = list(y2017)
holi_lst = []
for idx in [y2014, y2015, y2016, y2017] :
    for idx2 in col_lst :
        holi_lst.extend(list(idx[idx2]))

holi_lst = list(set(holi_lst))
holi_lst = holi_lst[1:]
holi_lst = set(list(map(lambda x : datetime.datetime.strptime(str(x), "%Y-%m-%d"), holi_lst)))

index = list(new_df.index)
for idx in index :
    interval = set([])
    date = new_df.loc[idx, "개봉일"]
    
    for day in range(0,14) :
        new_date = date + datetime.timedelta(days=day)
        interval.add(new_date)
        
        if day == 7 :
            new_df.loc[idx, "D+7 휴일수"] = 8 - len(interval - holi_lst)
        elif day == 13 :
            new_df.loc[idx, "D+13 휴일수"] = 14 - len(interval - holi_lst)

In [27]:
#개봉전 정보 처리
path_dir = "C:\\Users\\LG_\\Desktop\\와이빅타\\17-2 빅콘-영화분석\\Downloads"

In [28]:
#새로운 데이터 프레임 만들기
best_df = pd.DataFrame(columns=['영화명','개봉일', 'year', 'month', 'day', '대표국적', '제작사', '배급사', '수입사', '등급', '장르', '영화구분','runtime', '감독', '배우', 'D+7 휴일수', 'D+13 휴일수', '국산', '미국산', 'D-1 시사회수', 'D-5 시사회수',"naver_리뷰수", "naver_wanna_see", "naver_not_wanna_see", "naver_ratio"])
best_df = best_df.set_index("영화명")
best_df = pd.concat([best_df, new_df])

for idx in range(1,29) :
    for idx2 in ["순위", "매출액", "누적매출액", "관객수", "누적관객수", "스크린수", "좌석수", "상영횟수"] :
        best_df["D-" + str(idx) +" "+ idx2] = 0
       
        
for idx in [0, 1] :
    for idx2 in ["순위", "매출액", "누적매출액", "관객수", "누적관객수", "스크린수", "좌석수", "상영횟수"] :
        best_df["D+" + str(idx) +" "+ idx2] = 0
        
        
for idx in [7, 13] :
    best_df["D+" + str(idx) +" " + "누적관객수"] = 0

        

best_df.head(3)

,D+13 휴일수,D+7 휴일수,D-1 시사회수,D-5 시사회수,day,month,naver_not_wanna_see,naver_ratio,naver_wanna_see,naver_리뷰수,runtime,weekday,year,감독,개봉일,국산,대표국적,등급,미국산,배급사,배우,수입사,영화구분,장르,제작사,D-1 순위,D-1 매출액,D-1 누적매출액,D-1 관객수,D-1 누적관객수,D-1 스크린수,D-1 좌석수,D-1 상영횟수,D-2 순위,D-2 매출액,D-2 누적매출액,D-2 관객수,D-2 누적관객수,D-2 스크린수,D-2 좌석수,D-2 상영횟수,D-3 순위,D-3 매출액,D-3 누적매출액,D-3 관객수,D-3 누적관객수,D-3 스크린수,D-3 좌석수,D-3 상영횟수,D-4 순위,D-4 매출액,D-4 누적매출액,D-4 관객수,D-4 누적관객수,D-4 스크린수,D-4 좌석수,D-4 상영횟수,D-5 순위,D-5 매출액,D-5 누적매출액,D-5 관객수,D-5 누적관객수,D-5 스크린수,D-5 좌석수,D-5 상영횟수,D-6 순위,D-6 매출액,D-6 누적매출액,D-6 관객수,D-6 누적관객수,D-6 스크린수,D-6 좌석수,D-6 상영횟수,D-7 순위,D-7 매출액,D-7 누적매출액,D-7 관객수,D-7 누적관객수,D-7 스크린수,D-7 좌석수,D-7 상영횟수,D-8 순위,D-8 매출액,D-8 누적매출액,D-8 관객수,D-8 누적관객수,D-8 스크린수,D-8 좌석수,D-8 상영횟수,D-9 순위,D-9 매출액,D-9 누적매출액,D-9 관객수,D-9 누적관객수,D-9 스크린수,D-9 좌석수,D-9 상영횟수,D-10 순위,D-10 매출액,D-10 누적매출액,D-10 관객수,D-10 누적관객수,D-10 스크린수,D-10 좌석수,D-10 상영횟수,D-11 순위,D-11 매출액,D-11 누적매출액,D-11 관객수,D-11 누적관객수,D-11 스크린수,D-11 좌석수,D-11 상영횟수,D-12 순위,D-12 매출액,D-12 누적매출액,D-12 관객수,D-12 누적관객수,D-12 스크린수,D-12 좌석수,D-12 상영횟수,D-13 순위,D-13 매출액,D-13 누적매출액,D-13 관객수,D-13 누적관객수,D-13 스크린수,D-13 좌석수,D-13 상영횟수,D-14 순위,D-14 매출액,D-14 누적매출액,D-14 관객수,D-14 누적관객수,D-14 스크린수,D-14 좌석수,D-14 상영횟수,D-15 순위,D-15 매출액,D-15 누적매출액,D-15 관객수,D-15 누적관객수,D-15 스크린수,D-15 좌석수,D-15 상영횟수,D-16 순위,D-16 매출액,D-16 누적매출액,D-16 관객수,D-16 누적관객수,D-16 스크린수,D-16 좌석수,D-16 상영횟수,D-17 순위,D-17 매출액,D-17 누적매출액,D-17 관객수,D-17 누적관객수,D-17 스크린수,D-17 좌석수,D-17 상영횟수,D-18 순위,D-18 매출액,D-18 누적매출액,D-18 관객수,D-18 누적관객수,D-18 스크린수,D-18 좌석수,D-18 상영횟수,D-19 순위,D-19 매출액,D-19 누적매출액,D-19 관객수,D-19 누적관객수,D-19 스크린수,D-19 좌석수,D-19 상영횟수,D-20 순위,D-20 매출액,D-20 누적매출액,D-20 관객수,D-20 누적관객수,D-20 스크린수,D-20 좌석수,D-20 상영횟수,D-21 순위,D-21 매출액,D-21 누적매출액,D-21 관객수,D-21 누적관객수,D-21 스크린수,D-21 좌석수,D-21 상영횟수,D-22 순위,D-22 매출액,D-22 누적매출액,D-22 관객수,D-22 누적관객수,D-22 스크린수,D-22 좌석수,D-22 상영횟수,D-23 순위,D-23 매출액,D-23 누적매출액,D-23 관객수,D-23 누적관객수,D-23 스크린수,D-23 좌석수,D-23 상영횟수,D-24 순위,D-24 매출액,D-24 누적매출액,D-24 관객수,D-24 누적관객수,D-24 스크린수,D-24 좌석수,D-24 상영횟수,D-25 순위,D-25 매출액,D-25 누적매출액,D-25 관객수,D-25 누적관객수,D-25 스크린수,D-25 좌석수,D-25 상영횟수,D-26 순위,D-26 매출액,D-26 누적매출액,D-26 관객수,D-26 누적관객수,D-26 스크린수,D-26 좌석수,D-26 상영횟수,D-27 순위,D-27 매출액,D-27 누적매출액,D-27 관객수,D-27 누적관객수,D-27 스크린수,D-27 좌석수,D-27 상영횟수,D-28 순위,D-28 매출액,D-28 누적매출액,D-28 관객수,D-28 누적관객수,D-28 스크린수,D-28 좌석수,D-28 상영횟수,D+0 순위,D+0 매출액,D+0 누적매출액,D+0 관객수,D+0 누적관객수,D+0 스크린수,D+0 좌석수,D+0 상영횟수,D+1 순위,D+1 매출액,D+1 누적매출액,D+1 관객수,D+1 누적관객수,D+1 스크린수,D+1 좌석수,D+1 상영횟수,D+7 누적관객수,D+13 누적관객수
영화명,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
명량,4.0,2.0,NaN,NaN,30,7,NaN,NaN,NaN,NaN,128.0,2.0,2014,[김한민],2014-07-30,1,한국,15세이상관람가,0,[씨제이이앤엠(주)],"['최민식', '류승룡', '조진웅']",NaN,1,[사극],[(주)빅스톤픽쳐스],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
국제시장,5.0,2.0,NaN,NaN,17,12,NaN,NaN,NaN,NaN,126.0,2.0,2014,[윤제균],2014-12-17,1,한국,12세이상관람가,0,[씨제이이앤엠(주)],"['황정민', '김윤진', '오달수', '정진영']",NaN,1,[드라마],"[(주)제이케이필름, 씨제이이앤엠(주)]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
베테랑,5.0,2.0,NaN,NaN,5,8,NaN,Na

In [29]:
lst = list(best_df.index)
print("<진행상황>")

for idx in range(1, 12) :
    file_list = os.listdir(path_dir+"\\"+str(idx))
    file_list.sort()
    print(int(idx/11 *100), "%")
    
    for name in file_list :
       
        #데이터프레임 가져오기
        temp =  pd.read_html("Downloads/" + str(idx) + "/" + str(name),  index_col =None)
        temp = pd.DataFrame(temp[0])
        temp["날짜"] = temp["날짜"].map(lambda x : datetime.datetime.strptime(str(x), "%Y-%m-%d"))
        temp = temp.set_index("날짜")
        
        #영화제목 가져오기
        temp2 = open("Downloads/" + str(idx) + "/" + str(name), "r", encoding="utf=8")
        temp2_name = temp2.readlines()
        temp2.close()
        temp2_name = re.findall("'.+'", temp2_name[14])
        temp2_name = eval(temp2_name[0])
        
        #예외값에 대한 처리
        if temp2_name == "미녀와 야수" :
            if idx == 1 :
                temp2_name = temp2_name + "(2017)"
            else :
                temp2_name = temp2_name + "(2014)"
                
        if temp2_name == "라이프" :
            if idx == 3 :
                temp2_name = temp2_name + "(2017)"
            else :
                temp2_name = temp2_name + "(2015)"
                
        
        if temp2_name in lst :  
            
            date = new_df.loc[temp2_name, "개봉일"]
            flag = temp.index - date
            
            #개봉이전 1~28일
            temp3 = temp[flag.days <0]
            temp4 = pd.DataFrame([temp2_name], columns = ["영화명"])
            temp4 = temp4.set_index("영화명")
            
            count = 0
            before4_count = 0
            count_flag = True
        
            for idx2, row in temp3.iterrows() :
                count += 1
                day = idx2 - date
                string = "D-" + str(abs(int(day.days))) + " "
                
                #개봉 1/5일전 까지의 시사회수 및 누적
                if abs(int(day.days)) >= 5 :
                    before4_count = count
                        
                    temp4["D-5 누적매출액"] = temp3.loc[idx2, "누적매출액"]
                    temp4["D-5 누적관객수"] = temp3.loc[idx2, "누적관객수"]
                
                if abs(int(day.days)) >= 1 :
                    temp4["D-1 누적매출액"] = temp3.loc[idx2, "누적매출액"]
                    temp4["D-1 누적관객수"] = temp3.loc[idx2, "누적관객수"]
                    
                #개봉전의 데이터
                for col in ["순위", "매출액", "관객수", "스크린수", "좌석수", "상영횟수"] :
                    temp4[string+col] = temp3.loc[idx2, col]
            
            #개봉이후 0~1일
            temp7 = temp[(flag.days ==0)|(flag.days ==1)]
            temp8 = pd.DataFrame([temp2_name], columns = ["영화명"])
            temp8 = temp8.set_index("영화명")
            
            for idx2, row in temp7.iterrows() :
                day = idx2 - date
                string = "D+" + str(abs(int(day.days))) + " "
                
                for col in ["순위", "매출액", "누적매출액", "관객수", "누적관객수", "스크린수", "좌석수", "상영횟수"] :
                    temp8[string+col] = temp7.loc[idx2, col]
            
            #개봉이후 target
            temp5 = temp[(flag.days == 7) |  (flag.days ==13)]
            temp6 = pd.DataFrame([temp2_name], columns = ["영화명"])
            temp6 = temp6.set_index("영화명")
            
            for idx2, row in temp5.iterrows() :
                day = idx2 - date
                string = "D+" + str(abs(int(day.days))) + " "
                
                temp6[string+"누적관객수"] = temp5.loc[idx2, "누적관객수"]
            
            #업데이트
            best_df.update(pd.concat([temp4, temp6, temp8], axis=1))
            best_df.loc[temp2_name, "D-1 시사회수"] = count
            best_df.loc[temp2_name, "D-5 시사회수"] = before4_count
                
best_df.head(3)                

<진행상황>
9 %
18 %
27 %
36 %
45 %
54 %
63 %
72 %
81 %
90 %
100 %


,D+13 휴일수,D+7 휴일수,D-1 시사회수,D-5 시사회수,day,month,naver_not_wanna_see,naver_ratio,naver_wanna_see,naver_리뷰수,runtime,weekday,year,감독,개봉일,국산,대표국적,등급,미국산,배급사,배우,수입사,영화구분,장르,제작사,D-1 순위,D-1 매출액,D-1 누적매출액,D-1 관객수,D-1 누적관객수,D-1 스크린수,D-1 좌석수,D-1 상영횟수,D-2 순위,D-2 매출액,D-2 누적매출액,D-2 관객수,D-2 누적관객수,D-2 스크린수,D-2 좌석수,D-2 상영횟수,D-3 순위,D-3 매출액,D-3 누적매출액,D-3 관객수,D-3 누적관객수,D-3 스크린수,D-3 좌석수,D-3 상영횟수,D-4 순위,D-4 매출액,D-4 누적매출액,D-4 관객수,D-4 누적관객수,D-4 스크린수,D-4 좌석수,D-4 상영횟수,D-5 순위,D-5 매출액,D-5 누적매출액,D-5 관객수,D-5 누적관객수,D-5 스크린수,D-5 좌석수,D-5 상영횟수,D-6 순위,D-6 매출액,D-6 누적매출액,D-6 관객수,D-6 누적관객수,D-6 스크린수,D-6 좌석수,D-6 상영횟수,D-7 순위,D-7 매출액,D-7 누적매출액,D-7 관객수,D-7 누적관객수,D-7 스크린수,D-7 좌석수,D-7 상영횟수,D-8 순위,D-8 매출액,D-8 누적매출액,D-8 관객수,D-8 누적관객수,D-8 스크린수,D-8 좌석수,D-8 상영횟수,D-9 순위,D-9 매출액,D-9 누적매출액,D-9 관객수,D-9 누적관객수,D-9 스크린수,D-9 좌석수,D-9 상영횟수,D-10 순위,D-10 매출액,D-10 누적매출액,D-10 관객수,D-10 누적관객수,D-10 스크린수,D-10 좌석수,D-10 상영횟수,D-11 순위,D-11 매출액,D-11 누적매출액,D-11 관객수,D-11 누적관객수,D-11 스크린수,D-11 좌석수,D-11 상영횟수,D-12 순위,D-12 매출액,D-12 누적매출액,D-12 관객수,D-12 누적관객수,D-12 스크린수,D-12 좌석수,D-12 상영횟수,D-13 순위,D-13 매출액,D-13 누적매출액,D-13 관객수,D-13 누적관객수,D-13 스크린수,D-13 좌석수,D-13 상영횟수,D-14 순위,D-14 매출액,D-14 누적매출액,D-14 관객수,D-14 누적관객수,D-14 스크린수,D-14 좌석수,D-14 상영횟수,D-15 순위,D-15 매출액,D-15 누적매출액,D-15 관객수,D-15 누적관객수,D-15 스크린수,D-15 좌석수,D-15 상영횟수,D-16 순위,D-16 매출액,D-16 누적매출액,D-16 관객수,D-16 누적관객수,D-16 스크린수,D-16 좌석수,D-16 상영횟수,D-17 순위,D-17 매출액,D-17 누적매출액,D-17 관객수,D-17 누적관객수,D-17 스크린수,D-17 좌석수,D-17 상영횟수,D-18 순위,D-18 매출액,D-18 누적매출액,D-18 관객수,D-18 누적관객수,D-18 스크린수,D-18 좌석수,D-18 상영횟수,D-19 순위,D-19 매출액,D-19 누적매출액,D-19 관객수,D-19 누적관객수,D-19 스크린수,D-19 좌석수,D-19 상영횟수,D-20 순위,D-20 매출액,D-20 누적매출액,D-20 관객수,D-20 누적관객수,D-20 스크린수,D-20 좌석수,D-20 상영횟수,D-21 순위,D-21 매출액,D-21 누적매출액,D-21 관객수,D-21 누적관객수,D-21 스크린수,D-21 좌석수,D-21 상영횟수,D-22 순위,D-22 매출액,D-22 누적매출액,D-22 관객수,D-22 누적관객수,D-22 스크린수,D-22 좌석수,D-22 상영횟수,D-23 순위,D-23 매출액,D-23 누적매출액,D-23 관객수,D-23 누적관객수,D-23 스크린수,D-23 좌석수,D-23 상영횟수,D-24 순위,D-24 매출액,D-24 누적매출액,D-24 관객수,D-24 누적관객수,D-24 스크린수,D-24 좌석수,D-24 상영횟수,D-25 순위,D-25 매출액,D-25 누적매출액,D-25 관객수,D-25 누적관객수,D-25 스크린수,D-25 좌석수,D-25 상영횟수,D-26 순위,D-26 매출액,D-26 누적매출액,D-26 관객수,D-26 누적관객수,D-26 스크린수,D-26 좌석수,D-26 상영횟수,D-27 순위,D-27 매출액,D-27 누적매출액,D-27 관객수,D-27 누적관객수,D-27 스크린수,D-27 좌석수,D-27 상영횟수,D-28 순위,D-28 매출액,D-28 누적매출액,D-28 관객수,D-28 누적관객수,D-28 스크린수,D-28 좌석수,D-28 상영횟수,D+0 순위,D+0 매출액,D+0 누적매출액,D+0 관객수,D+0 누적관객수,D+0 스크린수,D+0 좌석수,D+0 상영횟수,D+1 순위,D+1 매출액,D+1 누적매출액,D+1 관객수,D+1 누적관객수,D+1 스크린수,D+1 좌석수,D+1 상영횟수,D+7 누적관객수,D+13 누적관객수
영화명,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
명량,4.0,2.0,9,6,30,7,NaN,NaN,NaN,NaN,128.0,2.0,2014,[김한민],2014-07-30,1,한국,15세이상관람가,0,[씨제이이앤엠(주)],"['최민식', '류승룡', '조진웅']",NaN,1,[사극],[(주)빅스톤픽쳐스],5.0,59633000.0,172231000.0,8073.0,22500.0,37.0,8120.0,40.0,11.0,13799000.0,0,1949.0,0,10.0,1949.0,12.0,9.0,16996000.0,0,2428.0,0,18.0,2627.0,18.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,10.0,10268000.0,81803000.0,1164.0,10050.0,6.0,1267.0,6.0,9.0,14979000.0,0,1890.0,0,8.0,1741.0,10.0,9.0,6696000.0,0,837.0,0,4.0,837.0,4.0,22.0,4416000.0,0,552.0,0,37.0,5600.0,38.0,7.0,44016000.0,0,5369.0,0,18.0,7241.0,28.0,34.0,1428000.0,0,238.0,0,1.0,238.0,1.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,1.0,4.708879e+09,4.881110e+09,682701.0,705201.0,1159.0,1152173.0,6147.0,1.0,5.243410e+09,1.012452e+10,705070.0,1410271.0,1202.0,1176034.0,6258.0,7329679.0,11775978.0
국제시장,5.0,2.0,20,17,17,12,NaN,NaN,NaN,NaN,126.0,2.0,2014,[윤제균],2014-1

In [30]:
#4일전/2일후 데이터로 만들어주기
#col만들기
best_df2 = best_df.copy()

for idx in [1,5] :
    for idx2 in ["상영횟수", "순위", "스크린수", "좌석수"] :
        string = "D-" + str(idx) + " " + idx2 + " 합"
        best_df2[string] = 0

for idx2 in ["상영횟수", "순위", "스크린수", "좌석수"] :
    string = "D+1 " + idx2 + " 합"
    best_df2[string] = 0    
    
#합 구하기
for idx in range(5, 29) :
    for idx2 in ["상영횟수", "순위", "스크린수", "좌석수"] :
        string = "D-" + str(idx) +" " +idx2
        result = "D-5 " + idx2 + " 합"
        best_df2[result] = best_df2[result] + best_df2[string]
        
for idx in range(1, 6) :
    for idx2 in ["상영횟수", "순위", "스크린수", "좌석수"] :
        string = "D-" + str(idx) +" " +idx2
        result = "D-1 " + idx2 + " 합"
        best_df2[result] = best_df2[result] + best_df2[string]
        
for idx in [0,1] :
    for idx2 in ["상영횟수", "순위", "스크린수", "좌석수"] :
        string = "D+" + str(idx) +" " +idx2
        result = "D+1 " + idx2 + " 합"
        best_df2[result] = best_df2[result] + best_df2[string]
        
#합 구하기   
for idx2 in ["상영횟수", "순위", "스크린수", "좌석수"] :
    string = "D-5 " + idx2 + " 합"
    result = "D-1 " + idx2 + " 합"
    best_df2[result] = best_df2[result] + best_df2[string]

#평균 구하기
for idx2 in ["상영횟수", "순위", "스크린수", "좌석수"] :
    result1 = "D-5 " + idx2 + " 평균"
    result2 = "D-1 " + idx2 + " 평균"
    result3 = "D+1 " + idx2 + " 평균"
    string1 = "D-5 " + idx2 + " 합"
    string2 = "D-1 " + idx2 + " 합"
    string3 = "D+1 " + idx2 + " 합"
    
    best_df2[result1] = best_df2[string1] / best_df2["D-5 시사회수"].map(lambda x : float("inf") if x == 0 else x)
    best_df2[result2] = best_df2[string2] / best_df2["D-1 시사회수"].map(lambda x : float("inf") if x == 0 else x)
    best_df2[result3] = best_df2[string3] / 2

for idx in ["관객수", "매출액"] :
    result1 = "D-5 " + idx + " 평균"
    result2 = "D-1 " + idx + " 평균"
    result3 = "D+1 " + idx + " 평균"
    
    best_df2[result1] = best_df2["D-5 누적" + idx] / best_df2["D-5 시사회수"].map(lambda x : float("inf") if x == 0 else x)
    best_df2[result2] = best_df2["D-1 누적" + idx] / best_df2["D-1 시사회수"].map(lambda x : float("inf") if x == 0 else x)
    best_df2[result3] = best_df2["D+1 누적" + idx] / 2
    

#합쳐진 데이터 지우기
lst = []
for idx in range(1, 29) :
    for idx2 in ["관객수", "매출액", "상영횟수", "순위", "스크린수", "좌석수"] :
        string = "D-" + str(idx) +" " +idx2
        lst.append(string)
        
    if idx != 1 and idx != 5 :
        for idx3 in ["누적관객수", "누적매출액"] :
            string = "D-" + str(idx) +" " +idx3
            lst.append(string)

lst2 = []
for idx in [0,1] :
    for idx2 in ["관객수", "매출액", "상영횟수", "순위", "스크린수", "좌석수"] :
        string = "D+" + str(idx) +" " +idx2
        lst2.append(string)
    
    if idx != 1 :
        for idx3 in ["누적관객수", "누적매출액"] :
            string = "D+" + str(idx) +" " +idx3
            lst2.append(string)

        
best_df2 = best_df2.drop(lst, axis=1)
best_df2 = best_df2.drop(lst2, axis=1)
best_df2 = best_df2.drop(["개봉일"], axis=1)
best_df2 = best_df2.drop(["D-5 순위 합", "D-1 순위 합"], axis=1)

best_df2.head(3)

,D+13 휴일수,D+7 휴일수,D-1 시사회수,D-5 시사회수,day,month,naver_not_wanna_see,naver_ratio,naver_wanna_see,naver_리뷰수,runtime,weekday,year,감독,국산,대표국적,등급,미국산,배급사,배우,수입사,영화구분,장르,제작사,D-1 누적매출액,D-1 누적관객수,D-5 누적매출액,D-5 누적관객수,D+1 누적매출액,D+1 누적관객수,D+7 누적관객수,D+13 누적관객수,D-1 상영횟수 합,D-1 스크린수 합,D-1 좌석수 합,D-5 상영횟수 합,D-5 스크린수 합,D-5 좌석수 합,D+1 상영횟수 합,D+1 순위 합,D+1 스크린수 합,D+1 좌석수 합,D-5 상영횟수 평균,D-1 상영횟수 평균,D+1 상영횟수 평균,D-5 순위 평균,D-1 순위 평균,D+1 순위 평균,D-5 스크린수 평균,D-1 스크린수 평균,D+1 스크린수 평균,D-5 좌석수 평균,D-1 좌석수 평균,D+1 좌석수 평균,D-5 관객수 평균,D-1 관객수 평균,D+1 관객수 평균,D-5 매출액 평균,D-1 매출액 평균,D+1 매출액 평균
영화명,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
명량,4.0,2.0,9,6,30,7,NaN,NaN,NaN,NaN,128.0,2.0,2014,[김한민],1,한국,15세이상관람가,0,[씨제이이앤엠(주)],"['최민식', '류승룡', '조진웅']",NaN,1,[사극],[(주)빅스톤픽쳐스],172231000.0,22500.0,81803000.0,10050.0,1.012452e+10,1410271.0,7329679.0,11775978.0,163.0,145.0,30887.0,87.0,74.0,16924.0,12405.0,2.0,2361.0,2328207.0,14.500000,18.111111,6202.5,15.166667,14.000000,1.0,12.333333,16.111111,1180.5,2820.666667,3431.888889,1164103.5,1675.000000,2500.000000,705135.5,1.363383e+07,1.913678e+07,5.062260e+09
국제시장,5.0,2.0,20,17,17,12,NaN,NaN,NaN,NaN,126.0,2.0,2014,[윤제균],1,한국,12세이상관람가,0,[씨제이이앤엠(주)],"['황정민', '김윤진', '오달수', '정진영']",NaN,1,[드라마],"[(주)제이케이필름, 씨제이이앤엠(주)]",256465000.0,33048.0,198916000.0,26194.0,3.069223e+09,416356.0,2312726.0,4869135.0,241.0,227.0,48722.0,191.0,178.0,38223.0,8935.0,3.0,1885.0,1603232.0,11.235294,12.050000,4467.5,26.529412,27.800000,1.5,10.470588,11.350000,942.5,2248.411765,2436.100000,801616.0,1540.823529,1652.400000,208178.0,1.170094e+07,1.282325e+07,1.534611e+09
베테랑,5.0,2.0,13,10,5,8,NaN,NaN,NaN,NaN,124.0,2.0,2015,[류승완],1,한국,15세이상관람가,0,[씨제이이앤엠(주)],"['황정민', '유아인', '유해진', '오달수']",NaN,1,[액션],[(주)외유내강],203522400.0,25421.0,143399000.0,18609.0,6.395053e+09,844547.0,3982844.0,7230136.0,154.0,147.0,30384.0,110.0,104.0,20920.0,10105.0,2.0,1899.0,1822415.0,11.000000,11.846154,5052.5,21.600000,20.153846,1.0,10.400000,11.307692,949.5,2092.000000,2337.230769,911207.5,1860.900000,1955.461538,422273.5,1.433990e+07,1.565557e+07,3.197527e+09


In [31]:
#네이버 리뷰
naver = pd.read_csv("movie_data/naver_movie.csv", encoding="CP949")

In [32]:
naver = naver.rename(columns={"name" : "영화명", "review_num_x" : "naver_리뷰수", "wanna_watch_x" : "naver_wanna_see" , "x_wanna_watch_x" :"naver_not_wanna_see" })
naver = naver[["영화명", "naver_리뷰수", "naver_wanna_see", "naver_not_wanna_see"]]
naver = naver.set_index("영화명")

naver["naver_리뷰수"] = naver["naver_리뷰수"].map(lambda x : int(str(x[:-1]).replace(",","")))
naver["naver_wanna_see"] = naver["naver_wanna_see"].map(lambda x : int(str(x).replace(",","")))
naver["naver_not_wanna_see"] = naver["naver_not_wanna_see"].map(lambda x : int(str(x).replace(",","")))
naver["naver_ratio"] = naver["naver_wanna_see"] / (naver["naver_wanna_see"]  + naver["naver_not_wanna_see"]) 
best_df2.update(naver)

In [33]:
#트위터 버즈량
#미녀와 야수(2017), 라이프(2015) 다시 크롤링
tweet1 = pd.read_csv("tweeter/tweeter_result/0-250tweet.csv", encoding='cp949')
tweet2 = pd.read_csv("tweeter/tweeter_result/hyunwoo.csv", encoding='cp949')
tweet3 = pd.read_csv("tweeter/tweeter_result/500~750.csv", encoding='cp949')
tweet4 = pd.read_csv("tweeter/tweeter_result/movietweet.csv", encoding='cp949')

tweet_lst = ['영화명', '단어영화포함트윗수1', '단어영화포함라잌수1', '단어영화포함트윗수2', '단어영화포함라잌수2']
tweet1 = tweet1[0:251][tweet_lst]
tweet2 = tweet2[251:501][tweet_lst]
tweet3 = tweet3[501:752][tweet_lst]
tweet4 = tweet4[752:][tweet_lst]

tweet = pd.concat([tweet1, tweet2, tweet3, tweet4])
tweet = tweet.set_index("영화명", drop=True)
tweet = tweet.rename(columns={'단어영화포함트윗수1' : 'after2_before28_tweet', '단어영화포함라잌수1' : 'after2_before28_like', '단어영화포함트윗수2' : 'before4_before34_tweet', '단어영화포함라잌수2' : 'before4_before34_like'})

best_df2['after2_before28_tweet'] = 0
best_df2['after2_before28_like'] = 0
best_df2['before4_before34_tweet'] = 0
best_df2['before4_before34_like'] = 0

best_df2.update(tweet)

In [34]:
#탑500 영화배우, 감독
top500 = pd.read_csv("movie_data/staff_list.csv", encoding= 'cp949')
top500 = top500.rename(columns={"0" : "영화명", "1" : "배우", "2" : "감독"}).drop("Unnamed: 0", axis = 1).set_index("영화명")
top500["배우"] = top500["배우"].map(lambda x : x.split(","))
top500["배우"] = top500["배우"].map(lambda x : list(map(str, x)))

temp_actor = list(top500["배우"].values.tolist())
top500_actors = list(set([val for sub_lst in temp_actor for val in sub_lst]))
top500_director = list(set(top500["감독"].values.tolist()))

best_df2["배우"] = best_df2["배우"].fillna("['']")
best_df2["배우"] = best_df2["배우"].map(lambda x: eval(x))
best_df2["top500_actors"] = best_df2["배우"].map(lambda x : len(set(x)) - len(set(x) - set(top500_actors)))
best_df2["top500_director"] = best_df2["감독"].map(lambda x : len(set(x)) - len(set(x) - set(top500_director)))

In [35]:
#배우/감독/배급사 점수화
#최근꺼일수록 가중치를 줌(1900년도 이후 날짜 x 관객수 /10억)
df = pd.read_csv("movie_data/col_point.csv", encoding = "cp949")

df = df.set_index("영화명")
df = df[["배우점수", "감독점수", "배급사점수"]]

best_df2["배우점수"] = 0
best_df2["감독점수"] = 0
best_df2["배급사점수"] = 0

best_df2.update(df)

In [40]:
#네이버 뉴스
best_df2["D-5 naver_news"] = 0
best_df2["D+1 naver_news"] = 0

news = pd.read_csv("movie_data/naver_news.csv", encoding = "cp949")
news.head()
news = news.rename(columns = {"0":"영화명", "28일전~4일전" : "D-5 naver_news", "개봉일~개봉2일후" : "D+1 naver_news"}).set_index("영화명")
news = news[["D-5 naver_news", "D+1 naver_news"]]
news["D+1 naver_news"] = news["D+1 naver_news"] + news["D-5 naver_news"]

best_df2.update(news)

In [42]:
#전처리 확인
print(best_df2.loc["국제시장", :])
best_df2.head(3)

D+13 휴일수                                       5
D+7 휴일수                                        2
D-1 시사회수                                      20
D-5 시사회수                                      17
day                                           17
month                                         12
naver_not_wanna_see                         2052
naver_ratio                             0.861688
naver_wanna_see                            12784
naver_리뷰수                                   1233
runtime                                      126
weekday                                        2
year                                        2014
감독                                         [윤제균]
국산                                             1
대표국적                                          한국
등급                                      12세이상관람가
미국산                                            0
배급사                                  [씨제이이앤엠(주)]
배우                          [황정민, 김윤진, 오달수, 정진영]
수입사                 

,D+13 휴일수,D+7 휴일수,D-1 시사회수,D-5 시사회수,day,month,naver_not_wanna_see,naver_ratio,naver_wanna_see,naver_리뷰수,runtime,weekday,year,감독,국산,대표국적,등급,미국산,배급사,배우,수입사,영화구분,장르,제작사,D-1 누적매출액,D-1 누적관객수,D-5 누적매출액,D-5 누적관객수,D+1 누적매출액,D+1 누적관객수,D+7 누적관객수,D+13 누적관객수,D-1 상영횟수 합,D-1 스크린수 합,D-1 좌석수 합,D-5 상영횟수 합,D-5 스크린수 합,D-5 좌석수 합,D+1 상영횟수 합,D+1 순위 합,D+1 스크린수 합,D+1 좌석수 합,D-5 상영횟수 평균,D-1 상영횟수 평균,D+1 상영횟수 평균,D-5 순위 평균,D-1 순위 평균,D+1 순위 평균,D-5 스크린수 평균,D-1 스크린수 평균,D+1 스크린수 평균,D-5 좌석수 평균,D-1 좌석수 평균,D+1 좌석수 평균,D-5 관객수 평균,D-1 관객수 평균,D+1 관객수 평균,D-5 매출액 평균,D-1 매출액 평균,D+1 매출액 평균,after2_before28_tweet,after2_before28_like,before4_before34_tweet,before4_before34_like,top500_actors,top500_director,배우점수,감독점수,배급사점수,D-5 naver_news,D+1 naver_news
영화명,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
명량,4.0,2.0,9,6,30,7,1126,0.923739,13639,1676,128.0,2.0,2014,[김한민],1,한국,15세이상관람가,0,[씨제이이앤엠(주)],"[최민식, 류승룡, 조진웅]",NaN,1,[사극],[(주)빅스톤픽쳐스],172231000.0,22500.0,81803000.0,10050.0,1.012452e+10,1410271.0,7329679.0,11775978.0,163.0,145.0,30887.0,87.0,74.0,16924.0,12405.0,2.0,2361.0,2328207.0,14.500000,18.111111,6202.5,15.166667,14.000000,1.0,12.333333,16.111111,1180.5,2820.666667,3431.888889,1164103.5,1675.000000,2500.000000,705135.5,1.363383e+07,1.913678e+07,5.062260e+09,6593.0,4713.0,5066.0,4464.0,3,1,47.909130,9.114424,98.955292,3633.0,4358.0
국제시장,5.0,2.0,20,17,17,12,2052,0.861688,12784,1233,126.0,2.0,2014,[윤제균],1,한국,12세이상관람가,0,[씨제이이앤엠(주)],"[황정민, 김윤진, 오달수, 정진영]",NaN,1,[드라마],"[(주)제이케이필름, 씨제이이앤엠(주)]",256465000.0,33048.0,198916000.0,26194.0,3.069223e+09,416356.0,2312726.0,4869135.0,241.0,227.0,48722.0,191.0,178.0,38223.0,8935.0,3.0,1885.0,1603232.0,11.235294,12.050000,4467.5,26.529412,27.800000,1.5,10.470588,11.350000,942.5,2248.411765,2436.100000,801616.0,1540.823529,1652.400000,208178.0,1.170094e+07,1.282325e+07,1.534611e+09,2896.0,691.0,1946.0,481.0,4,1,77.181236,8.307558,98.955292,3166.0,3546.0
베테랑,5.0,2.0,13,10,5,8,1339,0.88141,9952,688,124.0,2.0,2015,[류승완],1,한국,15세이상관람가,0,[씨제이이앤엠(주)],"[황정민, 유아인, 유해진, 오달수]",NaN,1,[액션],[(주)외유내강],203522400.0,25421.0,143399000.0,18609.0,6.395053e+09,844547.0,3982844.0,7230136.0,154.0,147.0,30384.0,110.0,104.0,20920.0,10105.0,2.0,1899.0,1822415.0,11.000000,11.846154,5052.5,21.600000,20.153846,1.0,10.400000,11.307692,949.5,2092.000000,2337.230769,911207.5,1860.900000,1955.461538,422273.5,1.433990e+07,1.565557e+07,3.197527e+09,2739.0,698.0,1115.0,401.0,4,1,94.237408,12.063382,98.955292,4066.0,4550.0


In [43]:
best_df2.to_csv("movie_preprocessing.csv")

In [109]:
#해야할 것 

#kobis에 데이터 언제 올라오는지 확인하기, 만약 대회제출 당일에 올라오지 않으면, 전날 데이터로 추정해야함

#겨울왕국 등 네이버 결과 크롤링 안된 것들이 있음

#네이버평점 크롤링
#네이버 뉴스 크롤링
#시리즈물 인지 확인하기, 이전 시리즈물의 흥행여부?
#정규화(kobis의 그래프 참조) -> kobis 공식통계 확인하기


#메이져 영화관에서 상영했는지 유무, 만약 사이즈가 큰 영화면, 큰 영화관에서 상영했을 듯, 이때 기간동안에 사영횟수가 증가했는지 여부모아, 몇번이나 상영했는지느를 알면 좋을 긋
#관객수/스크린수/좌석수/상영횟수/매출액 점유율 ->원래 데이터가 유리 ->개봉전은 무의미할 수도
#개봉후 등수
#서울 점유율?? ex) 서울시스크린/전체스크린 -> 젊은 사람의 선호를 반영할 수 있을 듯
#다른 나라는 거의 영화가 없는데 어케할지 ex) 일본, 프랑스 등
#상영요일이 중요한가? ex) 수요일
#개봉 후 몇 일 동안의 총 상영횟수/스크린수(?스크린수는 아마 중복될테니 평균으로)(누적도 고려)
#위아래 등수와의 차이 -> 점유율이 커버할 듯
#14~17에서 다양성 영화만 빼는 것은??
#서울 점유율을 넣어 말아???
#개봉 이후 점유율은 확실히 넣는 것이 좋고 개봉전은 무의미할 듯
#최소 몇명이 본걸 적용할지?
#3일 이상의 연휴(황금연휴가 있는지), 최대 연휴 길이 수



In [110]:
#리스트형 더미화
best_df3 = making_list_dummy(best_df2, ["제작사", "배급사", "수입사", "장르", "배우", "감독"])

In [111]:
best_df3.head(3)

,D+13 휴일수,D+7 휴일수,D-1 시사회수,D-5 시사회수,day,month,naver_not_wanna_see,naver_ratio,naver_wanna_see,naver_리뷰수,runtime,weekday,year,국산,대표국적,등급,미국산,영화구분,D-1 누적매출액,D-1 누적관객수,D-5 누적매출액,D-5 누적관객수,D+1 누적매출액,D+1 누적관객수,D+7 누적관객수,D+13 누적관객수,D-1 상영횟수 합,D-1 스크린수 합,D-1 좌석수 합,D-5 상영횟수 합,D-5 스크린수 합,D-5 좌석수 합,D+1 상영횟수 합,D+1 순위 합,D+1 스크린수 합,D+1 좌석수 합,D-5 상영횟수 평균,D-1 상영횟수 평균,D+1 상영횟수 평균,D-5 순위 평균,D-1 순위 평균,D+1 순위 평균,D-5 스크린수 평균,D-1 스크린수 평균,D+1 스크린수 평균,D-5 좌석수 평균,D-1 좌석수 평균,D+1 좌석수 평균,D-5 관객수 평균,D-1 관객수 평균,D+1 관객수 평균,D-5 매출액 평균,D-1 매출액 평균,D+1 매출액 평균,after2_before28_tweet,after2_before28_like,before4_before34_tweet,before4_before34_like,top500_actors,top500_director,배우점수,감독점수,배급사점수,제작사_ 두 도시 이야기 제작위원회,제작사_(유)내부자들 문화전문회사,제작사_(유)박열문화전문회사,제작사_(유)조이앤시네마,제작사_(유)조이앤컨텐츠그룹,제작사_(재)CBS,제작사_(주) 루스이소니도스,제작사_(주)26컴퍼니,제작사_(주)MBC,제작사_(주)고스트픽처스,제작사_(주)곽픽쳐스,제작사_(주)그린피쉬,제작사_(주)기린제작사,제작사_(주)김기덕 필름,제작사_(주)노마드필름,제작사_(주)누리픽쳐스,제작사_(주)다세포클럽,제작사_(주)다크서클픽쳐스,제작사_(주)담소필름,제작사_(주)더블유 팩토리,제작사_(주)더재미팩토리,제작사_(주)더타워픽쳐스,제작사_(주)데이드림 엔터테인먼트,제작사_(주)도서관옆스튜디오,제작사_(주)두타연,제작사_(주)드림써치C&C,제작사_(주)드림캡쳐,제작사_(주)디씨지플러스,제작사_(주)또하나의가족 제작위원회,제작사_(주)라인필름,제작사_(주)라임오렌지팩토리,제작사_(주)레드로버,제작사_(주)로드픽쳐스,제작사_(주)로제타시네마,제작사_(주)루이스 픽쳐스,제작사_(주)루카프로덕션,제작사_(주)률필름,제작사_(주)마음에드는영화사,제작사_(주)메이스엔터테인먼트,제작사_(주)메이저타운,제작사_(주)메이킹에이프린트,제작사_(주)명필름,제작사_(주)모베라픽처스,제작사_(주)모인그룹,제작사_(주)모호필름,제작사_(주)무브픽쳐스,제작사_(주)무비락,제작사_(주)무비스퀘어,제작사_(주)미디어캐슬,제작사_(주)미스터로맨스,제작사_(주)미인픽쳐스,제작사_(주)바른손이앤에이,제작사_(주)발렌타인필름,제작사_(주)뱅가드스튜디오,제작사_(주)버디필름,제작사_(주)베르디미디어,제작사_(주)보경사,제작사_(주)보리픽쳐스,제작사_(주)블러썸픽쳐스,제작사_(주)비에이엔터테인먼트,제작사_(주)비트윈픽처스,제작사_(주)빅스크리에이티브,제작사_(주)빅스토리픽쳐스,제작사_(주)빅스톤픽쳐스,제작사_(주)빅픽쳐,제작사_(주)사나이픽처스,제작사_(주)사이드미러,제작사_(주)삼거리픽쳐스,제작사_(주)삼지애니메이션,제작사_(주)샤인픽쳐스,제작사_(주)소넷엔터테인먼트,제작사_(주)쇼박스,제작사_(주)수필름,제작사_(주)스톰픽쳐스코리아,제작사_(주)스튜디오 애니멀,제작사_(주)스튜디오게일,제작사_(주)스튜디오후크,제작사_(주)시네마서비스,제작사_(주)시네마제니스,제작사_(주)신씨네,제작사_(주)싸이더스,제작사_(주)쏘울 크리에이티브,제작사_(주)씨네포트,제작사_(주)씨에이씨엔터테인먼트,제작사_(주)씨제이에스지 마당엔터테인먼트,제작사_(주)아우라씨엔씨,제작사_(주)아이언팩키지,제작사_(주)아이에이치큐,제작사_(주)아이엠티브이,제작사_(주)아이코닉스,제작사_(주)아지트필름,제작사_(주)아토,제작사_(주)알지엔터웍스,제작사_(주)앞에있다,제작사_(주)어나더썬데이,제작사_(주)어뮤즈엔터테인먼트,제작사_(주)어바웃필름,제작사_(주)얼리버드픽쳐스,제작사_(주)에스에이치기획,제작사_(주)에스엔케이픽처스,제작사_(주)에스엠 컬처앤콘텐츠,제작사_(주)에이디사공육,제작사_(주)에이치비엔터테인먼트,제작사_(주)에이트볼픽쳐스,제작사_(주)에코필름,제작사_(주)엘픽쳐스,제작사_(주)엠씨엠씨,제작사_(주)엠픽처스,제작사_(주)연우무대,제작사_(주)연필로명상하기,제작사_(주)영화나무,제작사_(주)영화사 금월,제작사_(주)영화사 담담,제작사_(주)영화사 북극곰,제작사_(주)영화사 빛나는 제국,제작사_(주)영화사 월광,제작사_(주)영화사 조제,제작사_(주)영화사 좋은날,제작사_(주)영화사AK기쁜우리젊은날,제작사_(주)영화사거미,제작사_(주)영화사꽃,제작사_(주)영화사동물의왕국,제작사_(주)영화사람,제작사_(주)영화사레드피터,제작사_(주)영화사봄,제작사_(주)영화사불,제작사_(주)영화사비단길,제작사_(주)영화사수박,제작사_(주)영화사수작,제작사_(주)영화사신세계,제작사_(주)영화사심플렉스,제작사_(주)영화사아이비젼,제작사_(주)영화사일취월장,제작사_(주)영화사장춘,제작사_(주)영화사하얼빈,제작사_(주)영화사호필름,제작사_(주)영화제작전원사,제작사_(주)예인플러스엔터테인먼트,제작사_(주)오에이엘,제작사_(주)오콘,제작사_(주)와이지엔터테인먼트,제작사_(주)외유내강,제작사_(주)용필름,제작사_(주)우주필름,제작사_(주)울림엔터테인먼트,제작사_(주)유니프로,제작사_(주)유비유필름,제작사_(주)율,제작사_(주)이디오플랜,제작사_(주)이스트스카이필름,제작사_(주)인벤트스톤,제작사_(주)인사이드픽쳐스,제작사_(주)전망좋은 영화사,제작사_(주)제이엠애니메이션,제작사_(주)제이케이필름,제작사_(주)제이콘컴퍼니,제작사_(주)조이래빗,제작사_(주)주피터필름,제작사_(주)지금이 아니면 안돼,제작사_(주)청우필름,제작사_(주)캠프비,제작사_(주)케이알씨지,제작사_(주)케이퍼필름,제작사_(주)케이프로덕션,제작사_(주)코리아스토리,제작사_(주)코브픽쳐스,제작사_(주)콘텐츠케이,제작사_(주)콘텐츠판다,제작사_(주)큐로홀딩스,제작사_(주)크레이지버드스튜디오,제작사_(주)크리픽쳐스,제작사_(주)키메이커,제작사_(주)타이거픽쳐스,제작사_(주)태원엔터테인먼트,제작사_(주)트리니티엔터테인먼트,제작사_(주)티피에스컴퍼니,제작사_(주)파파스필름,제작사_(주)판타지오픽쳐스,제작사_(주)팔레트픽처스,제작사_(주)팝콘에프앤엠,제작사_(주)퍼펙트스톰필름,제작사_(주)퍼플켓츠필름,제작사_(주)페퍼민트앤컴퍼니,제작사_(주)필마픽쳐스,제작사_(주)하리마오픽쳐스,제작사_(주)하이브미디어코프,제작사_(주)하이하버픽쳐스,제작사_(주)한이야기,제작사_(주)해무,제작사_(주)홍필름,제작사_(주)화인웍스,제작사_(주)화인컷,제작사_(주)효인엔터테인먼트,제작사_(합)알지애니메이션스튜디오,제작사_4.16 세월호 참사 시민기록위원회,제작사_60만번의트라이제작위원회,제작사_KBS,제작사_TCO(주)더콘텐츠온,제작사_㈜감독의집,제작사_㈜기억속의매미,제작사_㈜디지아트프로덕션,제작사_㈜리필름,제작사_㈜인디스토리,제작사_㈜희원엔터테인먼트,제작사_게릴라캠,제작사_골든타이드픽처스(주),제작사_광화문시네마,제작사_국가인권위원회,제작사_굿초이스컷픽쳐스(주),제작사_그린스톤픽처스,제작사_꼬마프레스,제작사_나우필름(주),제작사_다이스필름(주),제작사_더램프(주),제작사_더퀸디앤엠 주식회사,제작사_동우에이앤이(주),제작사_드림웍스 애니메이션,제작사_드림타워,제작사_라이브(주),제작사_라이온스 게이트 필름즈,제작사_레트로봇 주식회사,제작사_리 공동체 영화사,제작사_리드미컬그린(주),제작사_리양필름주식회사,제작사_리얼라이즈픽쳐스(주),제작사_명필름랩,제작사_모쿠슈라,제작사_무현,제작사_미디어길,제작사_민병훈필름,제작사_반달,제작사_반짝반짝영화사(주),제작사_버티고필름,제작사_베데코리아,제작사_베리굿스튜디오(주),제작사_볼,제작사_볼미디어(주),제작사_브릿지웍스 엔터테인먼트(주),제작사_블루블랙(주),제작사_비밀의화원,제작사_상상필름(주),제작사_상의원문화산업전문(유),제작사_스톤워크,제작사_스튜디오홀호

In [112]:
#명목형 더미화